In [ ]:
Content is user-generated and unverified.


Loading is taking longer than expected
There may be an issue with the content you’re trying to load.
The code itself may still be valid and functional.

#!/usr/bin/env python3
"""
Functional Equation Tester for Modular Forms

Tests whether a q-expansion satisfies the modular transformation property:
    f(γτ) = (cτ + d)^k f(τ)
for γ in Γ_0(N).

For weight k and level N, tests the key transformations and outputs
a score on a 0-100 scale.

Usage:
    python functional_equation_tester.py

Then modify the coeffs, k, and N variables below.
"""

import numpy as np
import mpmath as mp
from fractions import Fraction
from math import gcd, pi, sqrt

# Set high precision for mpmath
mp.dps = 50  # 50 decimal places

# ============================================================================
# USER INPUT SECTION - MODIFY THESE
# ============================================================================

# Your q-expansion coefficients [a_0, a_1, a_2, a_3, ...]
coeffs = [
    1, -5, -70/3, 28219/12, 1702183/30, 84674353/90, 
    223590851/63, -251901250975/1008, -203241992963123/22680, 
    -2549624882458823/16200
]

# Weight and level
k = 12  # weight
N = 1   # level

# Test parameters
num_test_points = 10  # How many τ values to test
max_terms = 50        # Maximum terms to use in q-expansion (limited by len(coeffs))

# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def q_from_tau(tau):
    """Compute q = e^{2πiτ} with high precision"""
    return mp.exp(2 * mp.pi * 1j * tau)

def evaluate_q_expansion(coeffs, q, max_terms=None):
    """
    Evaluate f(τ) = Σ a_n q^n where q = e^{2πiτ}
    
    Uses high precision arithmetic.
    """
    if max_terms is None:
        max_terms = len(coeffs)
    else:
        max_terms = min(max_terms, len(coeffs))
    
    result = mp.mpc(0)
    for n in range(max_terms):
        if n >= len(coeffs):
            break
        result += mp.mpc(coeffs[n]) * (q ** n)
    
    return result

def get_gamma_0_N_generators(N):
    """
    Get generators for Γ_0(N).
    
    Γ_0(N) = {[[a,b],[c,d]] in SL_2(Z) : c ≡ 0 (mod N)}
    
    Returns key test matrices:
    - S = [[0,-1],[1,0]] (not in Γ_0(N) for N>1, but related)
    - T = [[1,1],[0,1]] (in Γ_0(N))
    - W_N = [[0,-1],[N,0]] (Atkin-Lehner involution)
    """
    generators = []
    
    # T: translation by 1 (always in Γ_0(N))
    T = np.array([[1, 1], [0, 1]])
    generators.append(('T', T))
    
    # For level N = 1, also test S
    if N == 1:
        S = np.array([[0, -1], [1, 0]])
        generators.append(('S', S))
    
    # Atkin-Lehner involution W_N (when N is square-free)
    W_N = np.array([[0, -1], [N, 0]])
    generators.append((f'W_{N}', W_N))
    
    # Add some other elements of Γ_0(N)
    if N > 1:
        # [[1,0],[N,1]] is in Γ_0(N)
        generators.append(('[[1,0],[N,1]]', np.array([[1, 0], [N, 1]])))
    
    return generators

def apply_matrix_to_tau(matrix, tau):
    """
    Apply matrix [[a,b],[c,d]] to τ: τ ↦ (aτ + b)/(cτ + d)
    """
    a, b = int(matrix[0, 0]), int(matrix[0, 1])
    c, d = int(matrix[1, 0]), int(matrix[1, 1])
    
    return (mp.mpc(a) * tau + mp.mpc(b)) / (mp.mpc(c) * tau + mp.mpc(d))

def get_automorphy_factor(matrix, tau, k):
    """
    Compute (cτ + d)^k for matrix [[a,b],[c,d]] and weight k
    """
    c = int(matrix[1, 0])
    d = int(matrix[1, 1])
    return (mp.mpc(c) * tau + mp.mpc(d)) ** k

def test_transformation(coeffs, matrix, tau, k, max_terms, matrix_name="γ"):
    """
    Test if f(γτ) = (cτ + d)^k f(τ)
    
    Returns:
        - relative_error: |f(γτ) - (cτ+d)^k f(τ)| / |f(τ)|
        - lhs: value of f(γτ)
        - rhs: value of (cτ+d)^k f(τ)
    """
    # Evaluate f(τ)
    q = q_from_tau(tau)
    f_tau = evaluate_q_expansion(coeffs, q, max_terms)
    
    # Evaluate f(γτ)
    gamma_tau = apply_matrix_to_tau(matrix, tau)
    q_gamma = q_from_tau(gamma_tau)
    f_gamma_tau = evaluate_q_expansion(coeffs, q_gamma, max_terms)
    
    # Compute automorphy factor (cτ + d)^k
    automorphy = get_automorphy_factor(matrix, tau, k)
    
    # Expected value
    expected = automorphy * f_tau
    
    # Compute error
    if abs(f_tau) < 1e-30:
        # f(τ) too small, skip
        return None, f_gamma_tau, expected
    
    relative_error = abs(f_gamma_tau - expected) / abs(f_tau)
    
    return relative_error, f_gamma_tau, expected

def generate_test_points(num_points, imag_range=(0.5, 2.0), real_range=(-0.5, 0.5)):
    """
    Generate test points τ in the upper half-plane.
    Focus on the fundamental domain region.
    """
    test_points = []
    
    for i in range(num_points):
        # Random point in upper half-plane
        real_part = real_range[0] + (real_range[1] - real_range[0]) * (i / num_points)
        imag_part = imag_range[0] + (imag_range[1] - imag_range[0]) * ((i + 0.5) / num_points)
        
        tau = mp.mpc(real_part, imag_part)
        test_points.append(tau)
    
    return test_points

# ============================================================================
# MAIN TEST FUNCTION
# ============================================================================

def test_functional_equation(coeffs, k, N, num_test_points=10, max_terms=50):
    """
    Test functional equation for modular form of weight k and level N.
    
    Returns: (score, detailed_results)
        score: 0-100 based on how well functional equation is satisfied
    """
    print(f"\n{'='*70}")
    print(f"FUNCTIONAL EQUATION TEST")
    print(f"{'='*70}")
    print(f"Weight k = {k}")
    print(f"Level N = {N}")
    print(f"Number of coefficients: {len(coeffs)}")
    print(f"Using first {min(max_terms, len(coeffs))} terms")
    print(f"Testing at {num_test_points} points in upper half-plane")
    print()
    
    # Get transformation matrices to test
    generators = get_gamma_0_N_generators(N)
    
    print(f"Testing transformations:")
    for name, matrix in generators:
        print(f"  {name}: {matrix.tolist()}")
    print()
    
    # Generate test points
    test_points = generate_test_points(num_test_points)
    
    # Store all errors
    all_errors = []
    results_by_matrix = {}
    
    # Test each transformation
    for matrix_name, matrix in generators:
        print(f"\n{'─'*70}")
        print(f"Testing: {matrix_name}")
        print(f"{'─'*70}")
        
        errors_this_matrix = []
        
        for i, tau in enumerate(test_points):
            try:
                rel_error, lhs, rhs = test_transformation(
                    coeffs, matrix, tau, k, max_terms, matrix_name
                )
                
                if rel_error is not None:
                    errors_this_matrix.append(rel_error)
                    all_errors.append(rel_error)
                    
                    if i < 3:  # Show first 3 test points in detail
                        print(f"  τ = {complex(tau):.4f}")
                        print(f"    f({matrix_name}τ) = {complex(lhs):.6e}")
                        print(f"    (cτ+d)^k f(τ) = {complex(rhs):.6e}")
                        print(f"    Relative error: {float(rel_error):.6e}")
                
            except Exception as e:
                print(f"  Error at τ = {complex(tau):.4f}: {e}")
        
        # Summary for this matrix
        if errors_this_matrix:
            avg_error = float(sum(errors_this_matrix) / len(errors_this_matrix))
            max_error = float(max(errors_this_matrix))
            min_error = float(min(errors_this_matrix))
            
            print(f"\n  Summary for {matrix_name}:")
            print(f"    Average relative error: {avg_error:.6e}")
            print(f"    Max relative error: {max_error:.6e}")
            print(f"    Min relative error: {min_error:.6e}")
            
            results_by_matrix[matrix_name] = {
                'avg_error': avg_error,
                'max_error': max_error,
                'min_error': min_error,
                'num_tests': len(errors_this_matrix)
            }
    
    # Overall assessment
    print(f"\n{'='*70}")
    print(f"OVERALL RESULTS")
    print(f"{'='*70}")
    
    if not all_errors:
        print("⚠ No valid tests completed")
        return 0, {}
    
    overall_avg_error = float(sum(all_errors) / len(all_errors))
    overall_max_error = float(max(all_errors))
    
    print(f"Total tests: {len(all_errors)}")
    print(f"Average relative error: {overall_avg_error:.6e}")
    print(f"Maximum relative error: {overall_max_error:.6e}")
    print()
    
    # Compute score (0-100)
    # Score based on negative log of average error
    # error ~ 1e-10 → score ~ 100
    # error ~ 1e-6 → score ~ 60
    # error ~ 1e-2 → score ~ 20
    # error ~ 1 → score ~ 0
    
    if overall_avg_error > 0:
        # Score = max(0, min(100, 60 - 10 * log10(error)))
        # This gives: 1e-10 → 100, 1e-6 → 60, 1e-2 → 20, 1 → 0
        score = max(0, min(100, 60 - 10 * np.log10(overall_avg_error)))
    else:
        score = 100
    
    print(f"{'='*70}")
    print(f"SCORE: {score:.1f}/100")
    print(f"{'='*70}")
    print()
    
    # Interpretation
    if score >= 90:
        print("✓✓✓ EXCELLENT - Functional equation satisfied to high precision")
        print("    Strong evidence for modularity")
    elif score >= 70:
        print("✓✓ GOOD - Functional equation approximately satisfied")
        print("    Moderate evidence for modularity")
    elif score >= 50:
        print("⚠ FAIR - Some agreement with functional equation")
        print("    Weak evidence, may need more terms or higher precision")
    elif score >= 30:
        print("⚠ POOR - Significant deviations from functional equation")
        print("    Likely not modular with these parameters")
    else:
        print("✗ FAILED - Functional equation not satisfied")
        print("    Not modular with weight k={k}, level N={N}")
    
    print(f"{'='*70}")
    print()
    
    # Guidelines
    print("Interpretation guide:")
    print("  90-100: Error < 10^-9  → Excellent, almost certainly modular")
    print("  70-90:  Error ~ 10^-7  → Good, likely modular")
    print("  50-70:  Error ~ 10^-5  → Fair, possibly modular (try more terms)")
    print("  30-50:  Error ~ 10^-3  → Poor, probably not modular")
    print("  0-30:   Error > 10^-2  → Failed, definitely not modular")
    print()
    
    results = {
        'score': score,
        'overall_avg_error': overall_avg_error,
        'overall_max_error': overall_max_error,
        'num_tests': len(all_errors),
        'by_matrix': results_by_matrix
    }
    
    return score, results

# ============================================================================
# RUN THE TEST
# ============================================================================

if __name__ == "__main__":
    print("\n" + "="*70)
    print("FUNCTIONAL EQUATION TESTER FOR MODULAR FORMS")
    print("="*70)
    
    # Adjust max_terms based on available coefficients
    actual_max_terms = min(max_terms, len(coeffs))
    
    score, results = test_functional_equation(
        coeffs, k, N, 
        num_test_points=num_test_points,
        max_terms=actual_max_terms
    )
    
    # Save results option (optional)
    """
    import json
    with open('functional_equation_results.json', 'w') as f:
        # Convert to serializable format
        save_data = {
            'score': score,
            'weight': k,
            'level': N,
            'num_coeffs': len(coeffs),
            'results': {
                'overall_avg_error': results['overall_avg_error'],
                'overall_max_error': results['overall_max_error'],
                'num_tests': results['num_tests']
            }
        }
        json.dump(save_data, f, indent=2)
    print("Results saved to functional_equation_results.json")
    """
